## Objective
Parses a text file table that has reinsurance and risk adjustment data for a benefit year into a CSV.

### Preprocessing
step 1: copy into raw text file

step 2: delete header 

HIOS ID
HIOS INPUTTED
INSURANCE COMPANY
NAME STATE
REINSURANCE
PAYMENT
AMOUNT (OR
NOT ELIGIBLE)
HHS RISK
ADJUSTMENT
TRANSFER
AMOUNT
(INDIVIDUAL
MARKET,
INCLUDING
CATASTROPHIC)
HHS RISK
ADJUSTMENT
TRANSFERS
AMOUNT (SMALL
GROUP MARKET)

step 3: delete page numbers through the regexp ^RI RA.\*

step 4: remove blank lines through the regexp ^\n\n

step 5: find and replace "not eligible" with "N/E"

step 6: find and replace "\$ -" with "\$-"

step 7: find and replace "\$ " with "\$" (also works for step 6)

step 8: find and replace "N/A - MA Issuer" with "N/A_MA_Issuer" (dataframe has trouble parsing "-")

step 9: find and replace "N/A - Default Charge" with "N/A_DefaultCharge"

step 10: delete footnotes on the first page

In [6]:
import pandas as pd
# import re

In [7]:
#with open('2014-Benefit-Year-Edit.txt', 'r') as myfile:
with open('input/2015-Benefit-Year-Edit-Not-Updated.txt', 'r') as myfile:
    s = myfile.read()

In [8]:
split = str.split(s)
length = len(split)

In [9]:
storage = pd.DataFrame(columns=["HIOS ID", "HIOS INPUTTED INSURANCE COMPANY NAME", "STATE", "REINSURANCE PAYMENT AMOUNT (OR NOT ELIGIBLE)",
               "HHS RISK ADJUSTMENT TRANSFER AMOUNT (INDIVIDUAL MARKET, INCLUDING CATASTROPHIC)",
               "HHS RISK ADJUSTMENT TRANSFERS AMOUNT (SMALL GROUP MARKET)"])

# prog = re.compile('^[A-Z]{2}$')

i = 0
index = 0
while i < length - 1:
    hios_id = split[i]
    i = i + 1
    temp = i
    # while (prog.match(split[i]) == None or split[i] in ['US', 'BC', 'OF']): # look for state in quotes
    while (split[i] != 'N/E' and split[i][:1] != '$'):
        i = i + 1
    company_name = str(' '.join(split[temp:i - 1]))
    state = split[i - 1]
    reinsurance = split[i]
    i = i + 1
    individual = split[i]
    i = i + 1
    small_group = split[i] 
    i = i + 1
    storage.loc[index] = [hios_id, company_name, state, reinsurance, individual, small_group]
    index = index + 1
    
print(storage)

    HIOS ID              HIOS INPUTTED INSURANCE COMPANY NAME STATE  \
0     11082                      Aetna Life Insurance Company    AK   
1     38344          Premera Blue Cross Blue Shield of Alaska    AK   
2     38596                            Time Insurance Company    AK   
3     62637                 John Alden Life Insurance Company    AK   
4     73836                            Moda Health Plan, Inc.    AK   
5     80049               United Healthcare Insurance Company    AK   
6     13939                            Time Insurance Company    AL   
7     44580                          Humana Insurance Company    AL   
8     44902                Federated Mutual Insurance Company    AL   
9     46944             Blue Cross and Blue Shield of Alabama    AL   
10    49771                 John Alden Life Insurance Company    AL   
11    59809          United Healthcare Life Insurance Company    AL   
12    68259                United Healthcare of Alabama, Inc.    AL   
13    

In [10]:
# save it
storage.to_csv("output/2015-Benefit-Year-RI-RA-Not-Updated.csv")